# Isocompy Salar de Atacama application

We will explain an application of Isocompy in Salar de Atacama to clarify  the isocompy library usage on spatial and isotopic data to reach a group of models to estimate the spatio-temporal isotope values.
 
Isocompy is designed to reach a general estimation model when there is heterogeneity in number of available data between variables, or when not all measurments are available for all the observed samples (stations). The goal is to obtain seasonal isotopic models to be able to predict the isotope values in given time and space.



In this work, there is an acceptable amount of precipitation, relative humidity and temperature data in time ( the data doesn't have a good spatial variability), but  the isotopic observation data is limited and distributed in time intervals. More, the precipitation, relative humidity and temperature measurments are not available in the same time and/or space of the isotopic measurments.
To reach a general estimation of seasonal isotopic values, we will model generate a group of independent models for each of the precipitation, relative humidity and temperature, based on the spatial data for desired months. **We will call this group of meteorological models, stage 1 models.**



Then, because ther is not enough monthly data to have a meaningful isotopic monthly model, we will switch from monthly to seasonal models (in this case, a summer model for January, February and March): A seasonal isotopic model could contain isotope data from various month, integrated in a *season*. The precipitation, relative humidity and temperature values will be predicted in the coordination of isotopic measurments using stage 1 models, based on the date of isotopic observation date. Then the meteorological predicted values will be used to model isotopes. **We will call this group of meteorological models, stage 2 models.**

This work could be seperated to the following main steps:

1. Input data preparation and preprocessing


2. Constructing the meteorological models (stage 1)


3. Estimating the meteorological values based on the meteorological models in points where there is isotopic observations


4. Constructing the isotopic models (stage 2)


5. Predict the desired spatio-temporal isotope values, generating residual plots, meteoric lines plot and maps 




For more information about the example, refer to the [article]().

The input data could be found in [github library]().


In [1]:
#Import libraries
from os.path import join
import pandas as pd
from isocompy.data_preparation import preprocess
from isocompy.reg_model import model
from isocompy.tools import session, evaluation, stats, plots

## Input data

Input data have to be `Pandas Dataframe`.
The fields ***ID***, ***Value*** and ***Date*** have to exist in each input dataframe.

In [2]:
#importing data
dir_inp=r"directory of the input data folder"

#Temperature data
temp=pd.read_excel(join(dir_inp,"temp_monthly.xls"),sheet_name="temp_monthly",header=0,index_col=False,keep_default_na=True)
#relative humidity data
hum=pd.read_excel(join(dir_inp,"hum_monthly.xls"),sheet_name="hum_monthly",header=0,index_col=False,keep_default_na=True)
#precipitation data
rain=pd.read_excel(join(dir_inp,"rain_monthly.xls"),sheet_name="rain_monthly",header=0,index_col=False,keep_default_na=True)

#isotope data: They are not going to be used until stage 2 models.
data_file_iso = join(dir_inp,"Isotopes.xls")
iso_18 = pd.read_excel(data_file_iso,sheet_name="ISOT18O",header=0,index_col=False,keep_default_na=True)
iso_2h=pd.read_excel(data_file_iso,sheet_name="ISOT2H",header=0,index_col=False,keep_default_na=True)

## 1. Input data preparation and preprocessing



To prepare each group of the input data (meteorological (stage 1) models and isotope models (stage 2) ), we have to use `preprocess` class. This class allows us to integrate the data from daily or monthly format to average monthly. It is also possible to remove outliers using the existed methods.  In automatic outlier detection and removal and default values could be found in `help(preprocess.fit)`.


`fields` parameter determines which columns (fields) of the dataframe have to be considered as independent variables to explain the dependent variable. The dependent variable measurments are in ***Value*** column of the dataframe. `var_name` determines the given name to the dependent variable.

In [4]:
#data class
ddir=r"directory of the outputs data folder"
ddir_inp_classess=join(ddir,"input_data_classess")
#Precipitation
#Precipitation
preped_prc=preprocess()
preped_prc.fit(inp_var=rain,var_name="prc",fields=["CooX","CooY","CooZ"],remove_outliers=False,direc=ddir_inp_classess)

#Temperature
preped_tmp=preprocess()
preped_tmp.fit(inp_var=temp,var_name="tmp",fields=["CooX","CooY","CooZ"],remove_outliers=False,direc=ddir_inp_classess)

#Humidity
preped_hmd=preprocess()
preped_hmd.fit(inp_var=hum,var_name="hmd",fields=["CooX","CooY","CooZ"],remove_outliers=False,direc=ddir_inp_classess)

#isotopes
prep_st2_iso1=preprocess()
prep_st2_iso1.fit(inp_var=iso_18,var_name="iso_18",fields=["CooX","CooY","CooZ"],remove_outliers=False,direc=ddir_inp_classess)

prep_st2_iso2=preprocess()
prep_st2_iso2.fit(inp_var=iso_2h,var_name="iso_2h",fields=["CooX","CooY","CooZ"],remove_outliers=False,direc=ddir_inp_classess)

## 2. Constructing the meteorological models (stage 1)




To construct a group of models, the `model` class object have to be used.
`st1_fit` method creates the (stage 1) meteorological models. `var_cls_list` accepts a list of **preprocess** class objects that we want to model. Each `preprocess` class object will have it's own estimation model based on `preprocess` class determined input arguments. In other words, for each `preprocess` class, `model` class will constructs and stores models. `st1_model_month_list` is a list of months that determine for which months the models to be created. (There is a seperate model for each month)

In [ ]:
#stage 1 model
dir_outs_st1=join(ddir,"st1_all_month")
est_class=model()
est_class.st1_fit(var_cls_list=[preped_prc,preped_tmp,preped_hmd],
                    st1_model_month_list=[1,2,3],
                    direc=dir_outs_st1)

#stage 1 model plots
plots.best_estimator_plots(est_class,st2=False)
plots.partial_dep_plots(est_class,st2=False)

## 3. Estimating the meteorological values based on the meteorological models in points where there is isotopic observations

`st1_predict` is used to estimete the meteorological values using the stage 1 models that are already created. The predictions from `st1_predict` will be fed to the second stage models.


To calculate stage 1 predictions, independent variables that are selected based on the statistical tests needed to be available. This means that the used independent features in st1 have to be available in each st2 `preprocess` class (in this case: *prep_st2_iso1* and *prep_st2_iso2* have to have *CooX, CooY, CooZ* fields).
`cls_list` is a list of  `preprocess` classes that we want to have an independent model for each one of them in stage 2. (in this case: one model for *prep_st2_iso1* and another one for *prep_st2_iso2*)

The `st2_model_month_list` identifies the *months* that form a *season* in stage 2. Just the data from that specific months will be selected and predicted. obviously, the months in `st2_model_month_list` have to exist in `st1_model_month_list`.

In [ ]:
#stage 1 prediction
est_class.st1_predict(cls_list=[prep_st2_iso1,prep_st2_iso2],st2_model_month_list=[1,2,3])

#save stage 1 model object
est_class_dir=session.save(est_class,name='est_class_st1') 

## 4. Constructing the isotopic models (stage 2)
To construct the stage 2 models, the independent variables for each `preprocess` introduced class have to be determined in `st2_model_var_dict`. In this example, the independent variables for *is1* and *is2* are *CooX , CooY, CooZ, tmp, hmd, prc*. If `st2_model_var_dict` is None, all independent variables will be selected.

In [ ]:
#stage 2 model: The predicted features from st2 will be used:
#determine the input and output of the second stage model:
st2_model_var_dict={"iso_18":["CooX","CooY","CooZ","tmp","prc","hmd"],"iso_2h":["CooX","CooY","CooZ","tmp","prc","hmd"]}
args_dic={"feature_selection":"auto","vif_threshold":5, "vif_selection_pairs":[["CooZ","tmp"]], "correlation_threshold":0.87,"vif_corr":True,"p_val":0.05}

est_class.st2_fit(model_var_dict=st2_model_var_dict,args_dic=args_dic)

#save stage 2
est_class_st2_dir=session.save(est_class,name='est_class_st2') 

#stage 2 model plots
plots.best_estimator_plots(est_class,st1=False)
plots.partial_dep_plots(est_class,st1=False) 


## 5. Predict the desired spatio-temporal isotope values
`evaluation` class is used to estimate values from stage 2 models (isotope models) and to evaluate the results. The estimations serves two perposes:


1. To evaluate the predictions by comparing it with the real values, global and local meteorological lines and investigate residuals.
    
    `pred_inputs` determines the dataframe that is gonna be used for prediction and evaluation.  Using the `all_preds` dataframe from the `model` class (here, *est_class*) that contains all the predictions and choosing the needed variables that are needed for first stage models (here, *CooX, CooY, CooZ*) as `pred_inputs`, the `predict` function will estimate the stage 2 models (for *iso_18* and *iso_2h*). calling `isotopes_meteoline_plot` from `plots` class, creates the isotope meteoric line  and residuals plots. Note that to execute `isotopes_meteoline_plot` for evaluation correctly, the `pred_inputs` dataframe have to include *ID* and *month* fields (which by default is created already in `all_preds` dataframe). `obs_data` and `residplot` have to be set to `True` since the `pred_inputs` contains observed data and we desire to have the residuals plots.

2. To generate predictions in desired spatio temporal scenarios (As an example, to make a map of the points in an specific month)

    `pred_inputs` is the input dataframe that includes the needed variables that are needed for first stage models (here, *CooX, CooY, CooZ*). There is no need to *ID* and *month* fields since the points are not observation points. The `predict` function will estimate the stage 2 models (for *iso_18* and *iso_2h*). Calling `isotopes_meteoline_plot` from `plots` class, creates the isotope meteorological line plots, plotting all the estimations for `pred_inputs` points.

It is possible to generate the maps of the area in each month for desired features using `map_generator` method. The *ev_class_map* which is the evaluation class containing the predictions in desired points, identifies the points that will be included in the generated maps. By identifing a shape file, it is possible to add a background shape file to the .png maps. Defining the *observed_class_list* will result in showing the points that the data are measured on the map.

In [ ]:

dir_outs=r"directory"

#to predict in points of observed data:
#determine the points with needed features (from the model class)
pred_inputs=est_class.all_preds[["CooX","CooY","CooZ","month","ID"]].reset_index()

ev_class_obs=evaluation()
ev_class_obs.predict(est_class,pred_inputs=pred_inputs,direc=join(dir_outs,"preds_obs"))

#to compare 2 isotopes, resodual plots and meteoric lines :                     
plots.isotopes_meteoline_plot(ev_class_obs,est_class,iso_18=iso_18,iso_2h=iso_2h,var_list=['iso_18','iso_2h'],obs_data=True,residplot=True)
       
                     
#to predict in points to generate maps: (no observations)
#read points:
data_file = join(dir_inp,"x_y_z.xls")
pred_inputs_map=pd.read_excel(data_file,sheet_name="x_y_z",header=0,index_col=False,keep_default_na=True)
                     
ev_class_map=evaluation()
ev_class_map.predict(est_class,pred_inputs=pred_inputs_map,direc=join(dir_outs,"preds_map"))
#to compare 2 isotopes and meteoric lines:       
plots.isotopes_meteoline_plot(ev_class_map,est_class,var_list=['iso_18','iso_2h'])

# To generate the maps

opt_title_list=["Precipitation (mm)","Relative Humidity (%)", "Temperature ($^\circ$C)", f'\N{GREEK SMALL LETTER DELTA}\N{SUPERSCRIPT ONE}\N{SUPERSCRIPT EIGHT}O'+'(\u2030 VSMOW)']
feat_list=["prc","hmd","tmp","iso_18"]
observed_class_list=[preped_prc,preped_hmd,preped_tmp,prep_st2_iso1]

# to create .pngs with a shape file and an HTML interactive map
shp_file=r"directory of the shape files"
plots.map_generator(ev_class=ev_class_map,feat_list=feat_list,observed_class_list=observed_class_list,shp_file=shp_file,opt_title_list=opt_title_list)

# Summary of some capabilities of Isocompy classes amd methods

The time window could be from monthly average of each year to monthly average of all years. In the former, just the inputs have to be for an specific year
ability to report model uncertainty

## preprocess.fit

* daily or monthly data
* possibility to determine el nino, la nina or all year
* possibility to remove outliers based on quartiles or IQR formula. Sure it also can be done manually in the dataframe, then set "remove_outliers" to False
* Generate reports of data integration in each step
* Removing outliers could be done not considering the zero values in the database. (The zero values will be seperated, outliers will be removed, then zero values will be added to database)
* Data averaging method. available options are arithmetic  or geometric
* ability to define brute-force searching hyperparameters

## model.st1_fit

* Manual or auto feature selection. If auto: ability to determine p_value, VIF, VIF - correlation analysis and determining the possible pairs to remove if multicolinearity and correlation exists

## plots.best_estimator_plots

ability to plot and report the model fit and model uncertainty

## plots.partial_dep_plots

ability to plot partial dependency plots


## est_class.st1_predict
* first stage models integration
* the ability to limit the desired month

## model.st2_fit
* ability to select the best models based on the isotopic lines such as global or local.
* ability to select the best models based on various custom error functions.
* Manual or auto feature selection. If auto: ability to determine p_value, VIF, VIF - correlation analysis and determining the possible pairs to remove if multicolinearity and correlation exists

## plots.isotopes_meteoline_plot
* plots and reports the observes vs calculated or the new points against isotopic lines, calculating residuals and goodness of fits.

**all_preds** dataframe in evaluation class contains the predicted data from the first stage